In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

# 1. Prepare data (exclude targets from X, drop NA)
target = '1_month_recession_probability'  # or '3_month_recession_probability', etc.
X = df_fe.drop(columns=[target])
y = df_fe[target]

# 2. Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# 3. Train LightGBM for feature importance
lgb_train = lgb.Dataset(X_train, y_train)
params = {'objective': 'regression', 'verbosity': -1}
model = lgb.train(params, lgb_train, num_boost_round=100)

# 4. Get feature importances
importances = pd.Series(model.feature_importance(), index=X_train.columns)
importances = importances.sort_values(ascending=False)
print("Top 20 features by importance:")
print(importances.head(20))

# 5. Select top N features
N = 30  # or any number you want
top_features = importances.head(N).index.tolist()
X_reduced = X[top_features]

# 6. Save reduced feature set
df_reduced = df_fe[top_features + [target]]
df_reduced.to_csv('../data/processed/feature_engineered_economic_indicators_reduced.csv')
print(f"Saved reduced feature set with top {N} features to ../data/processed/feature_engineered_economic_indicators_reduced.csv")